In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from iexfinance.refdata import get_symbols, get_iex_symbols

import os
import sys
import pandas as pd
import altair as alt
import datetime as dt

from pandas import to_datetime
from datetime import datetime, date
from iexfinance.stocks import get_historical_data
from dateutil.relativedelta import relativedelta

In [3]:
# Used to log the `MESSAGES USED` in `_IEXBAse`.
# https://addisonlynch.github.io/iexfinance/stable/logging.html
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")
os.environ["IEX_LOG_LEVEL"] = "DEBUG"

DEBUG:root:test


In [4]:
IEX_TOKEN = "Tpk_57fa15c2c86b4dadbb31e0c1ad1db895"
os.environ["IEX_API_VERSION"] = "iexcloud-sandbox"
store = pd.HDFStore('dev_iex_store.h5')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [5]:
symbols = get_symbols(token=IEX_TOKEN)[:10]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sandbox.iexapis.com:443
DEBUG:urllib3.connectionpool:https://sandbox.iexapis.com:443 "GET /stable/ref-data/symbols?token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895 HTTP/1.1" 200 None


In [6]:
df = get_historical_data('AAPL', datetime.now() - relativedelta(days=60), to_datetime(dt.date.today()), close_only=True, output_format='pandas', token=IEX_TOKEN)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sandbox.iexapis.com:443
DEBUG:urllib3.connectionpool:https://sandbox.iexapis.com:443 "GET /stable/stock/market/batch?symbols=AAPL&types=chart&range=3m&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895 HTTP/1.1" 200 None


In [7]:
start = datetime.now() - relativedelta(days=60)
end = datetime.now() - relativedelta(days=1)
symbols = ['AAPL', 'GOOG']
json = get_historical_data(symbols, start, end, close_only=False, output_format='json', token=IEX_TOKEN)
df = get_historical_data(symbols, start, end, close_only=False, output_format='pandas', token=IEX_TOKEN)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sandbox.iexapis.com:443
DEBUG:urllib3.connectionpool:https://sandbox.iexapis.com:443 "GET /stable/stock/market/batch?symbols=AAPL%2CGOOG&types=chart&range=3m&chartByDay=False&chartCloseOnly=False&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sandbox.iexapis.com:443
DEBUG:urllib3.connectionpool:https://sandbox.iexapis.com:443 "GET /stable/stock/market/batch?symbols=AAPL%2CGOOG&types=chart&range=3m&chartByDay=False&chartCloseOnly=False&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895 HTTP/1.1" 200 None


In [8]:
pd.DataFrame.from_dict(json['AAPL']['chart']).columns

Index(['close', 'high', 'low', 'open', 'symbol', 'volume', 'id', 'key',
       'subkey', 'date', 'updated', 'changeOverTime', 'marketChangeOverTime',
       'uOpen', 'uClose', 'uHigh', 'uLow', 'uVolume', 'fOpen', 'fClose',
       'fHigh', 'fLow', 'fVolume', 'label', 'change', 'changePercent'],
      dtype='object')

In [9]:
def json_to_pandas(out):
    if len(symbols) > 1:
        out = {
            (symbol, day["date"]): day
            for symbol in out
            for day in out[symbol]["chart"]
        }
#         result = pd.DataFrame.from_dict(out, orient='index')
        result = pd.DataFrame.from_dict(out, orient="columns").drop("date").T
        result.index = result.index.set_levels(
            [result.index.levels[0], pd.to_datetime(result.index.levels[1])]
        )
        idx = pd.IndexSlice
        result = result.loc[idx[:, start : end], :]
    else:
        out = {entr["date"]: entr for entr in out[symbols[0]]["chart"]}
        result = pd.DataFrame.from_dict(out, orient="columns").drop("date").T
        result.index = pd.to_datetime(result.index)
        result = result.loc[start : end, :]
    return result

def json_to_pandas1(out):
    if len(symbols) > 1:
        out = {
            (symbol, pd.to_datetime(day["date"])): day
            for symbol in out
            for day in out[symbol]["chart"]
        }
        result = pd.DataFrame.from_dict(out, orient='index')
        idx = pd.IndexSlice
        result = result.loc[idx[:, start : end], :]
    else:
        out = {entr["date"]: entr for entr in out[symbols[0]]["chart"]}
        result = pd.DataFrame.from_dict(out, orient="columns").drop("date").T
        result.index = pd.to_datetime(result.index)
        result = result.loc[start : end, :]
    return result

In [10]:
import time

start_time = time.time()
json_to_pandas(json)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
json_to_pandas1(json)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.02695488929748535 seconds ---
--- 0.032715797424316406 seconds ---


In [13]:
all_symbols = get_symbols(token=IEX_TOKEN)
symbols_df = all_symbols[:10]
symbols = [symbol_metadata['symbol'] for _, symbol_metadata in symbols_df.iterrows()]
symbols

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sandbox.iexapis.com:443
DEBUG:urllib3.connectionpool:https://sandbox.iexapis.com:443 "GET /stable/ref-data/symbols?token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895 HTTP/1.1" 200 None


['A', 'AA', 'AAA', 'AAAU', 'AACG', 'AACQ', 'AACQW', 'AADR', 'AAIC', 'AAIC-B']